In [ ]:
# Import external resources
import json
from allennlp.common.util import import_submodules
from allennlp.models.archival import load_archive
from allennlp.predictors import Predictor
from collections import defaultdict
from typing import List

In [ ]:
# Change the working directory to be the root of the Github repo
# so that the module's code can be found by AllenNLP
import os
os.chdir('../..')
os.getcwd()
import_submodules('summarize')

In [ ]:
# Load the model and the predictor that will be used for the demo.
# Additionally, you can configure which decoding heuristics will be used
use_min_length = True
use_repeated_trigrams = True
use_length_penalty = True
use_coverage_penalty = False

nested_dict = lambda: defaultdict(nested_dict)
overrides = nested_dict()

# Remove PythonRouge dependency
overrides['model']['metrics'] = []

if not use_min_length:
    overrides['model']['beam_search']['min_steps'] = None
if not use_repeated_trigrams:
    overrides['model']['beam_search']['disallow_repeated_ngrams'] = None
    overrides['model']['beam_search']['repeated_ngrams_exceptions'] = None
if not use_length_penalty:
    overrides['model']['beam_search']['length_penalizer'] = None
if not use_coverage_penalty:
    overrides['model']['beam_search']['coverage_penalizer'] = None
    
archive = load_archive('https://danieldeutsch.s3.amazonaws.com/summarize/experiments/onmt/v1.0/pointer-generator/model/model.tar.gz',
                        overrides=json.dumps(overrides))
predictor = Predictor.from_archive(archive, 'sds-abstractive-predictor')

In [ ]:
# Define the method to produce the summary
def summarize(document: List[str]) -> str:
    output = predictor.predict_json({'document': document})
    return output['summary']

In [ ]:
# Define the document which should be summarized. The input should be
# tokenized with whitespace
document = [
    "( CNN ) The only thing crazier than a guy in snowbound Massachusetts boxing up the powdery white stuff and offering it for sale online ?",
    "People are actually buying it .",
    "For $ 89 , self-styled entrepreneur Kyle Waring will ship you 6 pounds of Boston-area snow in an insulated Styrofoam box -- enough for 10 to 15 snowballs , he says .",
    "But not if you live in New England or surrounding states .",
    "`` We will not ship snow to any states in the northeast ! '' says Waring 's website , ShipSnowYo.com . `` We 're in the business of expunging snow ! ''",
    "His website and social media accounts claim to have filled more than 133 orders for snow -- more than 30 on Tuesday alone , his busiest day yet .",
    "With more than 45 total inches , Boston has set a record this winter for the snowiest month in its history . Most residents see the huge piles of snow choking their yards and sidewalks as a nuisance , but Waring saw an opportunity .",
    "According to Boston.com , it all started a few weeks ago , when Waring and his wife were shoveling deep snow from their yard in Manchester-by-the-Sea , a coastal suburb north of Boston . He joked about shipping the stuff to friends and family in warmer states , and an idea was born .",
    "His business slogan : `` Our nightmare is your dream ! ''",
    "At first , ShipSnowYo sold snow packed into empty 16.9-ounce water bottles for $ 19.99 , but the snow usually melted before it reached its destination .",
    "So this week , Waring began shipping larger amounts in the Styrofoam cubes , which he promises will arrive anywhere in the U.S. in less than 20 hours .",
    "He also has begun selling a 10-pound box of snow for $ 119 .",
    "Many of his customers appear to be companies in warm-weather states who are buying the snow as a gag , he said .",
    "Whether Waring can sustain his gimmicky venture into the spring remains to be seen . But he has no shortage of product .",
    "`` At this rate , it 's going to be July until the snow melts , '' he told Boston.com . `` But I 've thought about taking this idea and running with it for other seasonal items . Maybe I 'll ship some fall foliage . ''"
]

# Run the summarization. If you are running this on MyBinder, it may take
# up to a minute to produce the summary
summarize(document)